经验证：
* 1、test的时间顺序并不是全部都是时序的，排行榜是按时序排列后取前0.78为public，后0.22为private，所以以后的工作重心将落到train的后44%
* 2、测试集中私有部分从2018-07-24 09:00:00开始，有9173472条数据，包含全部的building_id
* 3、测试集中公共部分与私有部分对应的数据从2017-07-24 09:00:00开始，有9174900条数据，包含全部的building_id
* 4、重点为训练集中2017-07-24 09:00:00后的数据建模，有9003109条数据，占训练集0.445
* 5、为防止测试集中不评分的那部分不知道算不算在0.22里面，也为防止一定的过拟合，可以考虑重点关注2017-07月后的训练数据，占0.51
* 6、验证EDA1后，发现0-104号建筑5月20日前的大量为0的数据是有误的，训练集不应该对这部分数据进行过拟合，删除这部分数据训练对提高public评分很有帮助，遗憾的是，private并不包含这类数据
* 7、猜想某些大范围为0的数据都同0-104号一样是有误的，重点验证7月后的该种数据，测试是否能提高public

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc


# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
import matplotlib.patches as patches

from plotly import tools, subplots
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.express as px
pd.set_option('max_columns', 100)


py.init_notebook_mode(connected=True)
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)
import plotly.graph_objs as go

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

../input/ashrae-energy-prediction/train.csv
../input/ashrae-energy-prediction/sample_submission.csv
../input/ashrae-energy-prediction/building_metadata.csv
../input/ashrae-energy-prediction/weather_train.csv
../input/ashrae-energy-prediction/weather_test.csv
../input/ashrae-energy-prediction/ashrae-energy-prediction.zip
../input/ashrae-energy-prediction/test.csv


In [2]:
# 导入数据, 只有train 和 test
train = pd.read_csv('../input/ashrae-energy-prediction/train.csv')
test = pd.read_csv('../input/ashrae-energy-prediction/test.csv')
building_df = pd.read_csv('../input/ashrae-energy-prediction/building_metadata.csv')

In [3]:
# 时间戳转换日期
train["timestamp"] = pd.to_datetime(train["timestamp"])
test["timestamp"] = pd.to_datetime(test["timestamp"])

In [4]:
train_exception = pd.read_pickle('../output/ashrae-eda-2/train_exception.pkl')
train['exception'] = train_exception.exception.values
del train_exception
gc.collect()

31

In [5]:
test_exception = pd.read_pickle('../output/ashrae-test-exception-label/test_exception.pkl')
test['exception'] = test_exception.exception.values
del test_exception
gc.collect()

20

In [6]:
# # 结合konstan的1.36的预测观察
# sub_136 = pd.read_csv('../input/ashrae-baseline-lgbm/submission.csv')
# test['meter_reading'] = sub_136.meter_reading.values
# del sub_136

# sub_test = pd.read_csv('../input/ashrae-baseline-lgbm-predict/submission.csv')
# test['meter_reading_test'] = sub_test.meter_reading.values
# del sub_test
# gc.collect()

In [7]:
gc.collect()

40

In [8]:
# num = 0
# for i in range(1449):
#     meter_array = train[train.building_id == i].meter.unique()
#     if len(meter_array) == 3:
#         if (1 in meter_array) and (2 in meter_array) and (3 in meter_array):
#             num += 1
# print(num)

In [9]:
# 1413个建筑有0，498个建筑有1，324个建筑有2，145个建筑有3
# [0,1]有132个，[0,2]64，[0,3]21, [1,2]13
# [0,1,2]220, [0,1,3]111
# [0,1,2,3]13

In [10]:
gc.collect()

60

In [11]:
# (0,0)到(104,0)从5月中旬才开始有读数, 其中有些在开始有读数之前有一些读数，不知什么情况
# 猜测该地区出现了大面积的电表故障或者大面积的电力故障，其中有极少建筑幸存或用于检修测试，所以有少量读数，5月中旬后修好才恢复该地区正常用电或读表

In [12]:
train.exception.value_counts(dropna=False)

 0    19506044
-1      388110
 1      156809
-2       85465
 2       79672
Name: exception, dtype: int64

In [13]:
test.exception.value_counts(dropna=False)

 0.0    38915560
 NaN     1365768
-1.0      773916
 1.0      312994
-2.0      170502
 2.0      158860
Name: exception, dtype: int64

In [14]:
def exception_label(bid, meter, start, end):
    print(f'标记bid={bid}, meter={meter}异常')
    train.loc[(train.building_id == bid)  & (train.meter == meter) & (train.timestamp >= start) & (train.timestamp <= end), 'exception'] = 1
    test.loc[(test.building_id == bid)  & (test.meter == meter) & (test.timestamp >= start.replace('2016', '2017')) & (test.timestamp <= end.replace('2016', '2017')), 'exception'] = 1
    test.loc[(test.building_id == bid)  & (test.meter == meter) & (test.timestamp >= start.replace('2016', '2018')) & (test.timestamp <= end.replace('2016', '2018')), 'exception'] = 1
    gc.collect()

In [15]:
## 异常标记

train.loc[(train.building_id == 7)  & (train.meter == 1) & (train.timestamp > '2016-11-03 10:00:00') & (train.timestamp < '2016-11-28 13:00:00'), 'exception'] = 1
test.loc[(test.building_id == 7)  & (test.meter == 1) & (test.timestamp > '2017-11-03 10:00:00') & (test.timestamp < '2017-11-28 13:00:00'), 'exception'] = 1
test.loc[(test.building_id == 7)  & (test.meter == 1) & (test.timestamp > '2018-11-03 10:00:00') & (test.timestamp < '2018-11-28 13:00:00'), 'exception'] = 1

train.loc[(train.building_id == 9)  & (train.meter == 1) & (train.timestamp > '2016-12-06 09:00:00'), 'exception'] = 1
test.loc[(test.building_id == 9)  & (test.meter == 1) & (test.timestamp > '2017-12-06 09:00:00') & (test.timestamp <= '2017-12-31 23:00:00'), 'exception'] = 1
test.loc[(test.building_id == 9)  & (test.meter == 1) & (test.timestamp > '2018-12-06 09:00:00'), 'exception'] = 1

train.loc[(train.building_id == 29) & (train.meter == 1) & (train.timestamp > '2016-11-05 12:00:00'), 'exception'] = 1
test.loc[(test.building_id == 29) & (test.meter == 1) & (test.timestamp > '2017-11-05 12:00:00') & (test.timestamp <= '2017-12-31 23:00:00'), 'exception'] = 1
test.loc[(test.building_id == 29) & (test.meter == 1) & (test.timestamp > '2018-11-05 12:00:00'), 'exception'] = 1

train.loc[(train.building_id == 75) & (train.meter == 1) & (train.timestamp > '2016-08-05 13:00:00') & (train.timestamp < '2016-08-26 10:00:00'), 'exception'] = 1
test.loc[(test.building_id == 75) & (test.meter == 1) & (test.timestamp > '2017-08-05 13:00:00') & (test.timestamp < '2017-08-26 10:00:00'), 'exception'] = 1
test.loc[(test.building_id == 75) & (test.meter == 1) & (test.timestamp > '2018-08-05 13:00:00') & (test.timestamp < '2018-08-26 10:00:00'), 'exception'] = 1

train.loc[(train.building_id == 95) & (train.meter == 1) & (train.timestamp > '2016-08-08 10:00:00') & (train.timestamp < '2016-08-25 13:00:00'), 'exception'] = 1
test.loc[(test.building_id == 95) & (test.meter == 1) & (test.timestamp > '2017-08-08 10:00:00') & (test.timestamp < '2017-08-25 13:00:00'), 'exception'] = 1
test.loc[(test.building_id == 95) & (test.meter == 1) & (test.timestamp > '2018-08-08 10:00:00') & (test.timestamp < '2018-08-25 13:00:00'), 'exception'] = 1

train.loc[(train.building_id == 96) & (train.meter == 1) & (train.timestamp > '2016-09-12 14:00:00') & (train.timestamp < '2016-09-26 15:00:00'), 'exception'] = 1
test.loc[(test.building_id == 96) & (test.meter == 1) & (test.timestamp > '2017-09-12 14:00:00') & (test.timestamp < '2017-09-26 15:00:00'), 'exception'] = 1
test.loc[(test.building_id == 96) & (test.meter == 1) & (test.timestamp > '2018-09-12 14:00:00') & (test.timestamp < '2018-09-26 15:00:00'), 'exception'] = 1

train.loc[(train.building_id == 97) & (train.meter == 1) & (train.timestamp > '2016-08-08 14:00:00') & (train.timestamp < '2016-08-25 14:00:00'), 'exception'] = 1
test.loc[(test.building_id == 97) & (test.meter == 1) & (test.timestamp > '2017-08-08 14:00:00') & (test.timestamp < '2017-08-25 14:00:00'), 'exception'] = 1
test.loc[(test.building_id == 97) & (test.meter == 1) & (test.timestamp > '2018-08-08 14:00:00') & (test.timestamp < '2018-08-25 14:00:00'), 'exception'] = 1

train.loc[(train.building_id == 162) & (train.meter == 1) & (train.timestamp > '2016-09-13 12:00:00') & (train.timestamp < '2016-10-10 09:00:00'), 'exception'] = 1
test.loc[(test.building_id == 162) & (test.meter == 1) & (test.timestamp > '2017-09-13 12:00:00') & (test.timestamp < '2017-10-10 09:00:00'), 'exception'] = 1
test.loc[(test.building_id == 162) & (test.meter == 1) & (test.timestamp > '2018-09-13 12:00:00') & (test.timestamp < '2018-10-10 09:00:00'), 'exception'] = 1

train.loc[(train.building_id == 163) & (train.meter == 1) & (train.timestamp > '2016-11-17 08:00:00') & (train.timestamp < '2016-11-21 06:00:00'), 'exception'] = 1
train.loc[(train.building_id == 163) & (train.meter == 1) & (train.timestamp > '2016-11-25 00:00:00') & (train.timestamp < '2016-12-16 11:00:00'), 'exception'] = 1
train.loc[(train.building_id == 163) & (train.meter == 1) & (train.timestamp > '2016-12-18 04:00:00') & (train.timestamp < '2016-12-23 03:00:00'), 'exception'] = 1
train.loc[(train.building_id == 163) & (train.meter == 1) & (train.timestamp > '2016-12-26 05:00:00') & (train.timestamp < '2016-12-31 05:00:00'), 'exception'] = 1

test.loc[(test.building_id == 163) & (test.meter == 1) & (test.timestamp > '2017-11-17 08:00:00') & (test.timestamp < '2017-11-21 06:00:00'), 'exception'] = 1
test.loc[(test.building_id == 163) & (test.meter == 1) & (test.timestamp > '2017-11-25 00:00:00') & (test.timestamp < '2017-12-16 11:00:00'), 'exception'] = 1
test.loc[(test.building_id == 163) & (test.meter == 1) & (test.timestamp > '2017-12-18 04:00:00') & (test.timestamp < '2017-12-23 03:00:00'), 'exception'] = 1
test.loc[(test.building_id == 163) & (test.meter == 1) & (test.timestamp > '2017-12-26 05:00:00') & (test.timestamp < '2017-12-31 05:00:00'), 'exception'] = 1

test.loc[(test.building_id == 163) & (test.meter == 1) & (test.timestamp > '2018-11-17 08:00:00') & (test.timestamp < '2018-11-21 06:00:00'), 'exception'] = 1
test.loc[(test.building_id == 163) & (test.meter == 1) & (test.timestamp > '2018-11-25 00:00:00') & (test.timestamp < '2018-12-16 11:00:00'), 'exception'] = 1
test.loc[(test.building_id == 163) & (test.meter == 1) & (test.timestamp > '2018-12-18 04:00:00') & (test.timestamp < '2018-12-23 03:00:00'), 'exception'] = 1
test.loc[(test.building_id == 163) & (test.meter == 1) & (test.timestamp > '2018-12-26 05:00:00') & (test.timestamp < '2018-12-31 05:00:00'), 'exception'] = 1


train.loc[(train.building_id == 180) & (train.meter == 1) & (train.timestamp > '2016-02-20 07:00:00') & (train.timestamp < '2016-04-05 12:00:00'), 'exception'] = 1
train.loc[(train.building_id == 180) & (train.meter == 1) & (train.timestamp > '2016-02-20 07:00:00') & (train.timestamp < '2016-04-05 12:00:00'), 'exception'] = 1
train.loc[(train.building_id == 180) & (train.meter == 1) & (train.timestamp > '2016-04-09 07:00:00') & (train.timestamp < '2016-10-19 23:00:00'), 'exception'] = 1
train.loc[(train.building_id == 180) & (train.meter == 1) & (train.timestamp > '2016-10-20 23:00:00') & (train.timestamp < '2016-12-31 22:00:00'), 'exception'] = 1

test.loc[(test.building_id == 180) & (test.meter == 1) & (test.timestamp > '2017-02-20 07:00:00') & (test.timestamp < '2017-04-05 12:00:00'), 'exception'] = 1
test.loc[(test.building_id == 180) & (test.meter == 1) & (test.timestamp > '2017-02-20 07:00:00') & (test.timestamp < '2017-04-05 12:00:00'), 'exception'] = 1
test.loc[(test.building_id == 180) & (test.meter == 1) & (test.timestamp > '2017-04-09 07:00:00') & (test.timestamp < '2017-10-19 23:00:00'), 'exception'] = 1
test.loc[(test.building_id == 180) & (test.meter == 1) & (test.timestamp > '2017-10-20 23:00:00') & (test.timestamp < '2017-12-31 22:00:00'), 'exception'] = 1

test.loc[(test.building_id == 180) & (test.meter == 1) & (test.timestamp > '2018-02-20 07:00:00') & (test.timestamp < '2018-04-05 12:00:00'), 'exception'] = 1
test.loc[(test.building_id == 180) & (test.meter == 1) & (test.timestamp > '2018-02-20 07:00:00') & (test.timestamp < '2018-04-05 12:00:00'), 'exception'] = 1
test.loc[(test.building_id == 180) & (test.meter == 1) & (test.timestamp > '2018-04-09 07:00:00') & (test.timestamp < '2018-10-19 23:00:00'), 'exception'] = 1
test.loc[(test.building_id == 180) & (test.meter == 1) & (test.timestamp > '2018-10-20 23:00:00') & (test.timestamp < '2018-12-31 22:00:00'), 'exception'] = 1

train.loc[(train.building_id == 188) & (train.meter == 1) & (train.timestamp > '2016-09-09 15:00:00') & (train.timestamp < '2016-11-07 13:00:00'), 'exception'] = 1
test.loc[(test.building_id == 188) & (test.meter == 1) & (test.timestamp > '2017-09-09 15:00:00') & (test.timestamp < '2017-11-07 13:00:00'), 'exception'] = 1
test.loc[(test.building_id == 188) & (test.meter == 1) & (test.timestamp > '2018-09-09 15:00:00') & (test.timestamp < '2018-11-07 13:00:00'), 'exception'] = 1

train.loc[(train.building_id == 189) & (train.meter == 1) & (train.timestamp > '2016-11-29 06:00:00') & (train.timestamp < '2016-12-13 14:00:00'), 'exception'] = 1
test.loc[(test.building_id == 189) & (test.meter == 1) & (test.timestamp > '2017-11-29 06:00:00') & (test.timestamp < '2017-12-13 14:00:00'), 'exception'] = 1
test.loc[(test.building_id == 189) & (test.meter == 1) & (test.timestamp > '2018-11-29 06:00:00') & (test.timestamp < '2018-12-13 14:00:00'), 'exception'] = 1

# 192自己学得还不错

In [16]:
exception_label(206, 1, '2016-11-30 16:00:00', '2016-12-14 14:00:00')

exception_label(207, 1, '2016-07-02 21:00:00', '2016-08-25 11:00:00')

# 231暂时不处理, 233不明

exception_label(235, 1, '2016-12-16 09:00:00', '2016-12-31 23:00:00')

# 直接预测还不错，不一定需要处理, 周围几个一样情况的也是
# exception_label(247, 1, '2016-08-08 07:00:00', '2016-08-16 08:00:00')
# exception_label(247, 1, '2016-10-28 07:00:00', '2016-11-01 12:00:00')

exception_label(252, 1, '2016-03-22 07:00:00', '2016-06-02 16:00:00')
exception_label(252, 1, '2016-08-19 08:00:00', '2016-11-16 11:00:00')
exception_label(252, 1, '2016-11-23 17:00:00', '2016-12-08 14:00:00')
exception_label(252, 1, '2016-12-16 17:00:00', '2016-12-29 14:00:00')

exception_label(258, 1, '2016-08-29 09:00:00', '2016-09-08 13:00:00')
exception_label(258, 1, '2016-09-19 03:00:00', '2016-12-11 13:00:00')

exception_label(259, 1, '2016-01-01 00:00:00', '2016-04-07 23:00:00')
exception_label(259, 1, '2016-04-15 00:00:00', '2016-05-07 02:00:00')
exception_label(259, 1, '2016-05-08 20:00:00', '2016-05-19 05:00:00')
exception_label(259, 1, '2016-05-20 09:00:00', '2016-06-08 10:00:00')
exception_label(259, 1, '2016-06-13 16:00:00', '2016-06-21 07:00:00')
exception_label(259, 1, '2016-10-12 20:00:00', '2016-10-24 13:00:00')
exception_label(259, 1, '2016-11-12 14:00:00', '2016-11-20 18:00:00')
exception_label(259, 1, '2016-11-23 17:00:00', '2016-12-15 23:00:00')

exception_label(260, 1, '2016-05-10 22:00:00', '2016-08-17 10:00:00')
exception_label(260, 1, '2016-08-28 04:00:00', '2016-10-31 12:00:00')

exception_label(263, 1, '2016-11-27 16:00:00', '2016-12-31 23:00:00')

exception_label(265, 1, '2016-03-11 13:00:00', '2016-03-21 08:00:00')
exception_label(265, 1, '2016-09-04 09:00:00', '2016-09-06 23:00:00') # 也许停了两天水？
exception_label(265, 1, '2016-09-15 06:00:00', '2016-09-20 19:00:00')
exception_label(265, 1, '2016-09-23 15:00:00', '2016-09-29 08:00:00')
exception_label(265, 1, '2016-10-03 19:00:00', '2016-10-08 22:00:00')
exception_label(265, 1, '2016-10-10 20:00:00', '2016-10-14 10:00:00') 

exception_label(267, 1, '2016-01-14 16:00:00', '2016-01-28 10:00:00') 
exception_label(267, 1, '2016-01-30 17:00:00', '2016-02-10 09:00:00') 
exception_label(267, 1, '2016-12-08 05:00:00', '2016-12-31 23:00:00') 

exception_label(268, 1, '2016-12-23 19:00:00', '2016-12-31 23:00:00') 

exception_label(284, 1, '2016-08-26 12:00:00', '2016-09-08 10:00:00') 

exception_label(290, 1, '2016-08-19 00:00:00', '2016-09-09 07:00:00') 
exception_label(290, 1, '2016-09-14 16:00:00', '2016-09-23 05:00:00') 
exception_label(290, 1, '2016-10-08 13:00:00', '2016-10-14 09:00:00') 

exception_label(745, 1, '2016-01-01 00:00:00', '2016-03-13 02:00:00') 
exception_label(745, 1, '2016-03-13 04:00:00', '2016-05-13 02:00:00')
exception_label(745, 1, '2016-10-22 04:00:00', '2016-12-31 23:00:00') 

exception_label(747, 1, '2016-01-01 00:00:00', '2016-01-18 11:00:00') 
exception_label(747, 1, '2016-01-19 13:00:00', '2016-03-06 11:00:00')
exception_label(747, 1, '2016-04-02 07:00:00', '2016-04-11 15:00:00') 
exception_label(747, 1, '2016-10-04 08:00:00', '2016-10-10 08:00:00') 
exception_label(747, 1, '2016-10-14 01:00:00', '2016-10-20 10:00:00') 
exception_label(747, 1, '2016-11-28 11:00:00', '2016-12-31 23:00:00')

exception_label(750, 1, '2016-10-04 09:00:00', '2016-10-10 11:00:00')

exception_label(752, 1, '2016-10-04 08:00:00', '2016-10-10 08:00:00')

exception_label(753, 1, '2016-01-01 00:00:00', '2016-01-31 15:00:00')
exception_label(753, 1, '2016-02-04 08:00:00', '2016-02-16 07:00:00')
exception_label(753, 1, '2016-04-02 23:00:00', '2016-04-13 14:00:00')
exception_label(753, 1, '2016-04-28 16:00:00', '2016-05-09 18:00:00')
exception_label(753, 1, '2016-05-15 05:00:00', '2016-05-19 14:00:00')
exception_label(753, 1, '2016-09-21 16:00:00', '2016-10-10 08:00:00')
exception_label(753, 1, '2016-12-03 15:00:00', '2016-12-18 05:00:00')
exception_label(753, 1, '2016-12-18 15:00:00', '2016-12-25 13:00:00')
exception_label(753, 1, '2016-12-27 21:00:00', '2016-12-31 23:00:00')

exception_label(763, 1, '2016-10-04 09:00:00', '2016-10-10 11:00:00')

exception_label(765, 1, '2016-01-01 00:00:00', '2016-01-18 09:00:00')
exception_label(765, 1, '2016-01-19 12:00:00', '2016-02-28 23:00:00')
exception_label(765, 1, '2016-04-01 22:00:00', '2016-04-22 16:00:00')
exception_label(765, 1, '2016-10-04 08:00:00', '2016-10-12 11:00:00')
exception_label(765, 1, '2016-11-09 21:00:00', '2016-11-25 07:00:00')

exception_label(765, 1, '2016-10-04 08:00:00', '2016-10-10 08:00:00') # 跟前面有几个很像，这几个一起停水了？

exception_label(769, 1, '2016-07-19 10:00:00', '2016-08-30 08:00:00')
exception_label(769, 1, '2016-08-30 10:00:00', '2016-10-04 01:00:00')
exception_label(769, 1, '2016-10-04 08:00:00', '2016-12-31 23:00:00')

exception_label(770, 1, '2016-10-04 09:00:00', '2016-10-10 11:00:00')

exception_label(774, 1, '2016-01-10 19:00:00', '2016-02-01 09:00:00')
exception_label(774, 1, '2016-10-04 09:00:00', '2016-11-01 10:00:00')
exception_label(774, 1, '2016-12-05 12:00:00', '2016-12-31 23:00:00')

exception_label(776, 1, '2016-10-04 08:00:00', '2016-10-10 08:00:00')

exception_label(777, 1, '2016-01-01 00:00:00', '2016-03-13 02:00:00')
exception_label(777, 1, '2016-03-13 04:00:00', '2016-04-18 08:00:00')
exception_label(777, 1, '2016-10-04 08:00:00', '2016-10-10 08:00:00')
exception_label(777, 1, '2016-11-22 09:00:00', '2016-11-29 17:00:00')
exception_label(777, 1, '2016-11-30 04:00:00', '2016-12-31 23:00:00')

exception_label(778, 1, '2016-01-01 00:00:00', '2016-09-08 09:00:00')
exception_label(778, 1, '2016-10-04 08:00:00', '2016-10-10 08:00:00')
exception_label(778, 1, '2016-10-19 09:00:00', '2016-12-31 23:00:00')

exception_label(780, 1, '2016-01-05 08:00:00', '2016-03-09 14:00:00')
exception_label(780, 1, '2016-09-30 04:00:00', '2016-12-31 23:00:00')

exception_label(786, 1, '2016-10-04 08:00:00', '2016-10-10 08:00:00')

exception_label(787, 1, '2016-01-01 00:00:00', '2016-03-08 11:00:00')
exception_label(787, 1, '2016-10-04 09:00:00', '2016-10-10 11:00:00')
exception_label(787, 1, '2016-12-02 15:00:00', '2016-12-31 23:00:00')

# 左右多0，暂时先不用
# exception_label(793, 1, '2016-01-01 00:00:00', '2016-05-06 12:00:00')
# exception_label(793, 1, '2016-11-04 18:00:00', '2016-12-31 23:00:00')

# exception_label(794, 1, '2016-01-01 00:00:00', '2016-05-15 21:00:00')
# exception_label(794, 1, '2016-10-24 09:00:00', '2016-12-31 23:00:00')

# exception_label(795, 1, '2016-01-01 00:00:00', '2016-05-16 21:00:00')
# exception_label(795, 1, '2016-10-18 16:00:00', '2016-12-31 23:00:00')

# exception_label(796, 1, '2016-11-21 10:00:00', '2016-12-31 23:00:00')

# exception_label(797, 1, '2016-01-01 00:00:00', '2016-05-05 09:00:00')
# exception_label(797, 1, '2016-10-18 23:00:00', '2016-12-31 23:00:00')

# exception_label(799, 1, '2016-01-01 00:00:00', '2016-05-09 14:00:00')
# exception_label(799, 1, '2016-10-17 14:00:00', '2016-11-03 12:00:00')
# exception_label(799, 1, '2016-11-03 14:00:00', '2016-12-31 23:00:00')

# exception_label(800, 1, '2016-01-01 00:00:00', '2016-01-07 08:00:00')
# exception_label(800, 1, '2016-01-07 11:00:00', '2016-04-17 13:00:00')
# exception_label(800, 1, '2016-04-18 00:00:00', '2016-05-11 11:00:00')
# exception_label(800, 1, '2016-10-18 23:00:00', '2016-11-09 16:00:00')
# exception_label(800, 1, '2016-10-18 23:00:00', '2016-11-09 16:00:00')
# exception_label(800, 1, '2016-11-09 18:00:00', '2016-12-12 12:00:00')
# exception_label(800, 1, '2016-11-09 18:00:00', '2016-12-12 12:00:00')
# exception_label(800, 1, '2016-12-12 14:00:00', '2016-12-31 23:00:00')

# exception_label(803, 1, '2016-01-01 00:00:00', '2016-02-17 16:00:00')
# exception_label(803, 1, '2016-02-17 19:00:00', '2016-03-19 08:00:00')
# exception_label(803, 1, '2016-03-19 11:00:00', '2016-04-04 04:00:00')
# exception_label(803, 1, '2016-04-04 07:00:00', '2016-04-17 16:00:00')
# exception_label(803, 1, '2016-10-20 01:00:00', '2016-11-04 14:00:00')
# exception_label(803, 1, '2016-11-16 20:00:00', '2016-11-25 07:00:00')
# exception_label(803, 1, '2016-12-05 13:00:00', '2016-12-13 09:00:00')
# exception_label(803, 1, '2016-12-13 15:00:00', '2016-12-21 16:00:00')
# exception_label(803, 1, '2016-12-21 18:00:00', '2016-12-31 23:00:00')

exception_label(874, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(874, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

exception_label(875, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(875, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

exception_label(878, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(878, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

exception_label(883, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(883, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

exception_label(885, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(885, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

exception_label(886, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(886, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

exception_label(887, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(887, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

exception_label(888, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(888, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

exception_label(889, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(889, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

exception_label(890, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(890, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

exception_label(891, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(891, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

exception_label(893, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(893, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')
exception_label(893, 1, '2016-09-01 12:00:00', '2016-09-09 07:00:00')

exception_label(894, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
exception_label(894, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

exception_label(911, 1, '2016-10-20 12:00:00', '2016-11-02 14:00:00')

exception_label(929, 1, '2016-02-23 23:00:00', '2016-03-16 00:00:00')

exception_label(954, 1, '2016-01-01 00:00:00', '2016-08-08 10:00:00')

exception_label(957, 1, '2016-03-23 14:00:00', '2016-05-18 12:00:00')

exception_label(961, 1, '2016-08-29 16:00:00', '2016-09-08 12:00:00')

exception_label(978, 1, '2016-07-29 14:00:00', '2016-09-06 08:00:00')
exception_label(978, 1, '2016-10-24 10:00:00', '2016-12-16 13:00:00')

for bid in [895, 896, 898, 899, 900, 901, 903, 905, 906, 907, 908, 910, 911, 912, 913, 914, 915, 916, 917, 918, 920, 921, 922, 923, 924, 925, 926, 927, 928,
           929, 931, 932, 933, 934, 935, 942, 945, 946, 948, 949, 951, 952, 953, 954, 955, 956, 957, 958, 959, 960, 961, 962, 963, 964, 965, 966, 967, 968, 
           969, 971, 972, 973, 974, 975, 976, 978, 979, 980, 981, 983, 987, 989, 990, 991, 992, 993, 994, 995, 996, 997]:
    exception_label(bid, 1, '2016-07-01 15:00:00', '2016-07-05 05:00:00')
    exception_label(bid, 1, '2016-10-14 23:00:00', '2016-10-17 07:00:00')

标记bid=206, meter=1异常
标记bid=207, meter=1异常
标记bid=235, meter=1异常
标记bid=252, meter=1异常
标记bid=252, meter=1异常
标记bid=252, meter=1异常
标记bid=252, meter=1异常
标记bid=258, meter=1异常
标记bid=258, meter=1异常
标记bid=259, meter=1异常
标记bid=259, meter=1异常
标记bid=259, meter=1异常
标记bid=259, meter=1异常
标记bid=259, meter=1异常
标记bid=259, meter=1异常
标记bid=259, meter=1异常
标记bid=259, meter=1异常
标记bid=260, meter=1异常
标记bid=260, meter=1异常
标记bid=263, meter=1异常
标记bid=265, meter=1异常
标记bid=265, meter=1异常
标记bid=265, meter=1异常
标记bid=265, meter=1异常
标记bid=265, meter=1异常
标记bid=265, meter=1异常
标记bid=267, meter=1异常
标记bid=267, meter=1异常
标记bid=267, meter=1异常
标记bid=268, meter=1异常
标记bid=284, meter=1异常
标记bid=290, meter=1异常
标记bid=290, meter=1异常
标记bid=290, meter=1异常
标记bid=745, meter=1异常
标记bid=745, meter=1异常
标记bid=745, meter=1异常
标记bid=747, meter=1异常
标记bid=747, meter=1异常
标记bid=747, meter=1异常
标记bid=747, meter=1异常
标记bid=747, meter=1异常
标记bid=747, meter=1异常
标记bid=750, meter=1异常
标记bid=752, meter=1异常
标记bid=753, meter=1异常
标记bid=753, meter=1异常
标记bid=753, me

In [17]:
# exception_label(998, 1, '2016-01-01 00:00:00', '2016-03-18 10:00:00')
# exception_label(998, 1, '2016-12-05 10:00:00', '2016-12-31 23:00:00')

# exception_label(1000, 1, '2016-01-01 00:00:00', '2016-04-08 16:00:00')
# exception_label(1000, 1, '2016-04-23 15:00:00', '2016-05-03 13:00:00')
# exception_label(1000, 1, '2016-09-09 01:00:00', '2016-09-16 11:00:00')
# exception_label(1000, 1, '2016-10-28 15:00:00', '2016-11-03 16:00:00')
# exception_label(1000, 1, '2016-11-15 16:00:00', '2016-11-20 15:00:00')
# exception_label(1000, 1, '2016-11-20 17:00:00', '2016-11-26 15:00:00')
# exception_label(1000, 1, '2016-11-26 23:00:00', '2016-12-31 23:00:00')

# exception_label(1001, 1, '2016-01-01 00:00:00', '2016-03-04 06:00:00')
# exception_label(1001, 1, '2016-03-05 14:00:00', '2016-03-10 10:00:00')
# exception_label(1001, 1, '2016-03-12 11:00:00', '2016-03-21 04:00:00')
# exception_label(1001, 1, '2016-03-21 19:00:00', '2016-04-02 11:00:00')
# exception_label(1001, 1, '2016-11-19 15:00:00', '2016-12-31 23:00:00')

# exception_label(1013, 1, '2016-01-01 00:00:00', '2016-03-18 11:00:00')
# exception_label(1013, 1, '2016-09-28 20:00:00', '2016-10-12 02:00:00')
# exception_label(1013, 1, '2016-11-20 02:00:00', '2016-12-02 23:00:00')
# exception_label(1013, 1, '2016-12-04 07:00:00', '2016-12-11 07:00:00')
# exception_label(1013, 1, '2016-12-19 15:00:00', '2016-12-25 16:00:00')

exception_label(1017, 1, '2016-01-01 00:00:00', '2016-03-18 10:00:00')
exception_label(1017, 1, '2016-10-19 08:00:00', '2016-10-30 07:00:00')
exception_label(1017, 1, '2016-11-28 16:00:00', '2016-12-31 23:00:00')

exception_label(1018, 1, '2016-01-01 00:00:00', '2016-11-08 16:00:00')
exception_label(1018, 1, '2016-11-08 18:00:00', '2016-12-31 23:00:00')

# exception_label(1022, 1, '2016-01-01 00:00:00', '2016-02-29 09:00:00')
train.loc[(train.building_id == 1022) & (train.meter == 1) & (train.timestamp >= '2016-01-01 00:00:00') & (train.timestamp <= '2016-02-29 09:00:00'), 'exception'] = 1
test.loc[(test.building_id == 1022) & (test.meter == 1) & (test.timestamp >= '2017-01-01 00:00:00') & (test.timestamp <= '2017-02-28 23:00:00'), 'exception'] = 1
test.loc[(test.building_id == 1022) & (test.meter == 1) & (test.timestamp >= '2018-01-01 00:00:00') & (test.timestamp <= '2018-02-28 23:00:00'), 'exception'] = 1
exception_label(1022, 1, '2016-03-09 19:00:00', '2016-04-23 15:00:00')
exception_label(1022, 1, '2016-04-26 07:00:00', '2016-12-31 23:00:00')

# exception_label(1028, 1, '2016-01-01 00:00:00', '2016-05-25 11:00:00')
# exception_label(1028, 1, '2016-10-18 14:00:00', '2016-12-31 23:00:00')

# exception_label(1030, 1, '2016-01-01 00:00:00', '2016-05-18 12:00:00')
# exception_label(1030, 1, '2016-06-08 09:00:00', '2016-06-13 12:00:00')
# exception_label(1030, 1, '2016-10-11 10:00:00', '2016-12-31 23:00:00')

# exception_label(1031, 1, '2016-01-01 00:00:00', '2016-05-20 10:00:00')
# exception_label(1031, 1, '2016-06-08 06:00:00', '2016-06-13 12:00:00')
# exception_label(1031, 1, '2016-10-11 10:00:00', '2016-12-31 23:00:00')

# exception_label(1032, 1, '2016-01-01 00:00:00', '2016-05-25 08:00:00')
# exception_label(1032, 1, '2016-10-12 16:00:00', '2016-12-31 23:00:00')

# exception_label(1074, 1, '2016-01-01 00:00:00', '2016-04-13 09:00:00')
# exception_label(1074, 1, '2016-11-22 11:00:00', '2016-12-31 23:00:00')

# exception_label(1077, 1, '2016-01-01 00:00:00', '2016-03-29 10:00:00')
# exception_label(1077, 1, '2016-11-14 21:00:00', '2016-12-31 23:00:00')

# exception_label(1078, 1, '2016-11-29 12:00:00', '2016-12-06 11:00:00')
# exception_label(1078, 1, '2016-12-07 18:00:00', '2016-12-19 16:00:00')
# exception_label(1078, 1, '2016-12-24 17:00:00', '2016-12-28 10:00:00')

# exception_label(1079, 1, '2016-01-01 00:00:00', '2016-04-12 10:00:00')
# exception_label(1079, 1, '2016-04-25 08:00:00', '2016-05-02 09:00:00')
# exception_label(1079, 1, '2016-11-17 15:00:00', '2016-12-31 23:00:00')

# exception_label(1080, 1, '2016-01-01 00:00:00', '2016-04-14 19:00:00')
# exception_label(1080, 1, '2016-11-22 07:00:00', '2016-12-31 23:00:00')

# exception_label(1084, 1, '2016-01-01 00:00:00', '2016-03-16 10:00:00')
# exception_label(1084, 1, '2016-10-22 15:00:00', '2016-12-31 23:00:00')

# exception_label(1085, 1, '2016-01-01 00:00:00', '2016-04-13 14:00:00')
# exception_label(1085, 1, '2016-11-19 13:00:00', '2016-12-31 23:00:00')

# exception_label(1090, 1, '2016-01-01 00:00:00', '2016-02-27 11:00:00')
# exception_label(1090, 1, '2016-11-18 07:00:00', '2016-11-28 06:00:00')
# exception_label(1090, 1, '2016-11-28 15:00:00', '2016-12-31 23:00:00')

# exception_label(1093, 1, '2016-01-01 00:00:00', '2016-04-29 15:00:00')
# exception_label(1093, 1, '2016-07-18 20:00:00', '2016-07-27 13:00:00')
# exception_label(1093, 1, '2016-10-24 15:00:00', '2016-12-31 23:00:00')

# exception_label(1094, 1, '2016-01-01 00:00:00', '2016-04-14 10:00:00')
# exception_label(1094, 1, '2016-11-13 19:00:00', '2016-12-31 23:00:00')

# exception_label(1095, 1, '2016-01-01 00:00:00', '2016-04-14 14:00:00')
# exception_label(1095, 1, '2016-04-23 22:00:00', '2016-05-03 12:00:00')
# exception_label(1095, 1, '2016-10-25 15:00:00', '2016-12-31 23:00:00')

# exception_label(1097, 1, '2016-01-01 00:00:00', '2016-03-08 15:00:00')
# exception_label(1097, 1, '2016-03-12 19:00:00', '2016-03-20 13:00:00')
# exception_label(1097, 1, '2016-11-26 20:00:00', '2016-12-05 14:00:00')

# exception_label(1098, 1, '2016-01-01 00:00:00', '2016-04-19 13:00:00')
# exception_label(1098, 1, '2016-04-21 19:00:00', '2016-05-02 10:00:00')
# exception_label(1098, 1, '2016-10-18 19:00:00', '2016-12-31 23:00:00')

# exception_label(1101, 1, '2016-01-01 00:00:00', '2016-04-14 12:00:00')
# exception_label(1101, 1, '2016-11-18 09:00:00', '2016-12-31 23:00:00')

# exception_label(1104, 1, '2016-01-01 00:00:00', '2016-03-09 10:00:00')
# exception_label(1104, 1, '2016-11-18 13:00:00', '2016-12-31 23:00:00')

# exception_label(1109, 1, '2016-01-01 00:00:00', '2016-04-13 13:00:00')
# exception_label(1109, 1, '2016-10-23 13:00:00', '2016-12-31 23:00:00') # 左右为0太多了，暂时先做这些，如果后续确实有效果，慢慢补充完整

exception_label(1130, 1, '2016-01-01 00:00:00', '2016-09-30 12:00:00')

exception_label(1160, 1, '2016-01-01 00:00:00', '2016-10-14 09:00:00')

exception_label(1223, 1, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1225, 1, '2016-08-23 00:00:00', '2016-10-11 13:00:00')

exception_label(1226, 1, '2016-08-23 00:00:00', '2016-10-20 12:00:00')

exception_label(1230, 1, '2016-08-22 09:00:00', '2016-10-05 17:00:00')

exception_label(1232, 1, '2016-01-06 17:00:00', '2016-01-10 15:00:00')
exception_label(1232, 1, '2016-01-10 21:00:00', '2016-01-16 14:00:00')
exception_label(1232, 1, '2016-06-23 17:00:00', '2016-08-31 19:00:00')

exception_label(1233, 1, '2016-09-28 04:00:00', '2016-10-20 12:00:00')
exception_label(1233, 1, '2016-11-09 16:00:00', '2016-12-31 23:00:00')

exception_label(1234, 1, '2016-09-28 04:00:00', '2016-10-20 12:00:00')
exception_label(1234, 1, '2016-11-09 16:00:00', '2016-12-31 23:00:00')

exception_label(1244, 1, '2016-07-13 14:00:00', '2016-08-31 19:00:00')

exception_label(1263, 1, '2016-11-10 14:00:00', '2016-11-21 14:00:00')

exception_label(1264, 1, '2016-08-22 18:00:00', '2016-09-22 19:00:00')
exception_label(1264, 1, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1265, 1, '2016-01-01 00:00:00', '2016-04-19 12:00:00')
exception_label(1265, 1, '2016-10-28 13:00:00', '2016-12-31 23:00:00')

exception_label(1272, 1, '2016-01-01 00:00:00', '2016-01-31 18:00:00')
exception_label(1272, 1, '2016-02-01 22:00:00', '2016-02-20 15:00:00')
exception_label(1272, 1, '2016-02-21 03:00:00', '2016-02-28 16:00:00')
exception_label(1272, 1, '2016-03-02 05:00:00', '2016-03-07 16:00:00')
exception_label(1272, 1, '2016-08-18 18:00:00', '2016-08-24 10:00:00')
exception_label(1272, 1, '2016-08-25 21:00:00', '2016-08-30 18:00:00')
exception_label(1272, 1, '2016-09-28 08:00:00', '2016-10-20 12:00:00')
exception_label(1272, 1, '2016-11-19 21:00:00', '2016-11-24 17:00:00')
exception_label(1272, 1, '2016-12-01 05:00:00', '2016-12-12 19:00:00')

exception_label(1273, 1, '2016-01-01 00:00:00', '2016-01-10 18:00:00')
exception_label(1273, 1, '2016-01-10 20:00:00', '2016-01-31 18:00:00')
exception_label(1273, 1, '2016-02-02 17:00:00', '2016-02-20 15:00:00')
exception_label(1273, 1, '2016-02-21 03:00:00', '2016-02-28 16:00:00')
exception_label(1273, 1, '2016-05-31 15:00:00', '2016-06-16 23:00:00')
exception_label(1273, 1, '2016-12-01 05:00:00', '2016-12-12 19:00:00')

exception_label(1274, 1, '2016-01-01 00:00:00', '2016-01-10 18:00:00')
exception_label(1274, 1, '2016-01-10 20:00:00', '2016-01-31 18:00:00')
exception_label(1274, 1, '2016-02-01 17:00:00', '2016-02-20 15:00:00')
exception_label(1274, 1, '2016-12-01 13:00:00', '2016-12-12 19:00:00')
exception_label(1274, 1, '2016-12-22 22:00:00', '2016-12-31 23:00:00')

exception_label(1275, 1, '2016-01-01 00:00:00', '2016-01-31 18:00:00')
exception_label(1275, 1, '2016-02-01 22:00:00', '2016-02-20 15:00:00')
exception_label(1275, 1, '2016-12-01 09:00:00', '2016-12-12 19:00:00')

# exception_label(1280, 1, '2016-02-29 00:00:00', '2016-05-02 04:00:00')
train.loc[(train.building_id == 1280) & (train.meter == 1) & (train.timestamp >= '2016-02-29 00:00:00') & (train.timestamp <= '2016-05-02 04:00:00'), 'exception'] = 1
test.loc[(test.building_id == 1280) & (test.meter == 1) & (test.timestamp >= '2017-03-01 00:00:00') & (test.timestamp <= '2017-05-02 04:00:00'), 'exception'] = 1
test.loc[(test.building_id == 1280) & (test.meter == 1) & (test.timestamp >= '2018-03-01 00:00:00') & (test.timestamp <= '2018-05-02 04:00:00'), 'exception'] = 1
exception_label(1280, 1, '2016-05-18 01:00:00', '2016-05-26 08:00:00')
exception_label(1280, 1, '2016-06-15 14:00:00', '2016-07-07 05:00:00')
exception_label(1280, 1, '2016-06-15 14:00:00', '2016-07-07 05:00:00')
exception_label(1280, 1, '2016-10-17 04:00:00', '2016-10-29 09:00:00')
exception_label(1280, 1, '2016-12-12 21:00:00', '2016-12-24 03:00:00')

# 1283感觉很有问题

exception_label(1288, 1, '2016-07-07 16:00:00', '2016-08-10 18:00:00')

exception_label(1291, 1, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1292, 1, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1293, 1, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1294, 1, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1295, 1, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1296, 1, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1297, 1, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1300, 1, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1302, 1, '2016-09-28 08:00:00', '2016-10-20 12:00:00')

exception_label(1308, 1, '2016-12-17 06:00:00', '2016-12-31 23:00:00')

exception_label(1310, 1, '2016-12-17 03:00:00', '2016-12-31 23:00:00')

exception_label(1317, 1, '2016-10-06 14:00:00', '2016-10-11 13:00:00')
exception_label(1317, 1, '2016-10-12 16:00:00', '2016-10-21 10:00:00')

标记bid=1017, meter=1异常
标记bid=1017, meter=1异常
标记bid=1017, meter=1异常
标记bid=1018, meter=1异常
标记bid=1018, meter=1异常
标记bid=1022, meter=1异常
标记bid=1022, meter=1异常
标记bid=1130, meter=1异常
标记bid=1160, meter=1异常
标记bid=1223, meter=1异常
标记bid=1225, meter=1异常
标记bid=1226, meter=1异常
标记bid=1230, meter=1异常
标记bid=1232, meter=1异常
标记bid=1232, meter=1异常
标记bid=1232, meter=1异常
标记bid=1233, meter=1异常
标记bid=1233, meter=1异常
标记bid=1234, meter=1异常
标记bid=1234, meter=1异常
标记bid=1244, meter=1异常
标记bid=1263, meter=1异常
标记bid=1264, meter=1异常
标记bid=1264, meter=1异常
标记bid=1265, meter=1异常
标记bid=1265, meter=1异常
标记bid=1272, meter=1异常
标记bid=1272, meter=1异常
标记bid=1272, meter=1异常
标记bid=1272, meter=1异常
标记bid=1272, meter=1异常
标记bid=1272, meter=1异常
标记bid=1272, meter=1异常
标记bid=1272, meter=1异常
标记bid=1272, meter=1异常
标记bid=1273, meter=1异常
标记bid=1273, meter=1异常
标记bid=1273, meter=1异常
标记bid=1273, meter=1异常
标记bid=1273, meter=1异常
标记bid=1273, meter=1异常
标记bid=1274, meter=1异常
标记bid=1274, meter=1异常
标记bid=1274, meter=1异常
标记bid=1274, meter=1异常
标记bid=1274

In [18]:
gc.collect()

20

In [19]:
%%time
for bid in train[(train.exception == 1) & (train.meter == 1)].building_id.unique():
    train.loc[(train.building_id == bid) & (train.meter == 1) & (train.exception == 0), 'exception'] = -1
    test.loc[(test.building_id == bid) & (test.meter == 1) & (test.exception != 1), 'exception'] = -1

CPU times: user 1min 20s, sys: 1min 1s, total: 2min 21s
Wall time: 2min 21s


In [20]:
train.exception.value_counts(dropna=False)

 0    18065316
-1     1670842
 1      314805
-2       85465
 2       79672
Name: exception, dtype: int64

In [21]:
test.exception.value_counts(dropna=False)

 0.0    36041924
-1.0     3388094
 NaN     1296044
 1.0      642176
-2.0      170502
 2.0      158860
Name: exception, dtype: int64

In [22]:
test[(test.building_id == 53) & (test.meter == 0)].exception.value_counts(dropna=False)

 NaN    14198
 1.0     3086
-1.0      236
Name: exception, dtype: int64

In [23]:
test.loc[(test.building_id == 53) & (test.meter == 0) & (test.exception.isna()), 'exception'] = 1

In [24]:
test[(test.building_id == 53) & (test.meter == 0)].exception.value_counts(dropna=False)

 1.0    17284
-1.0      236
Name: exception, dtype: int64

In [25]:
# ## 存疑异常标记

# # 30前期有大量0值
# train.loc[(train.building_id == 30) & (train.meter == 1) & (train.meter_reading < 212), 'exception'] = 2

# # 43大量0值
# train.loc[(train.building_id == 43) & (train.meter == 1) & (train.meter_reading == 0), 'exception'] = 2

# # 28大量0值
# train.loc[(train.building_id == 28) & (train.meter == 1) & (train.meter_reading == 0), 'exception'] = 2

In [26]:
gc.collect()

80

In [27]:
train[['exception']].to_pickle('../output/ashrae-eda-exception-label1/train_exception.pkl')
test[['exception']].to_pickle('../output/ashrae-eda-exception-label1/test_exception.pkl')

In [28]:
test.exception.value_counts(dropna=False)

 0.0    36041924
-1.0     3388094
 NaN     1281846
 1.0      656374
-2.0      170502
 2.0      158860
Name: exception, dtype: int64

In [29]:
for i in range(105):
    print(str(i) + ":" + str(train[train.building_id == i].meter.unique()))

0:[0]
1:[0]
2:[0]
3:[0]
4:[0]
5:[0]
6:[0]
7:[0 1]
8:[0]
9:[0 1]
10:[0]
11:[0]
12:[0]
13:[0 1]
14:[0 1]
15:[0 1]
16:[0]
17:[0]
18:[0]
19:[0]
20:[0]
21:[0]
22:[0]
23:[0]
24:[0]
25:[0]
26:[0]
27:[0]
28:[0 1]
29:[0 1]
30:[0 1]
31:[0 1]
32:[0]
33:[0]
34:[0]
35:[0]
36:[0]
37:[0]
38:[0]
39:[0]
40:[0]
41:[0]
42:[0]
43:[0 1]
44:[0]
45:[0]
46:[0]
47:[0]
48:[0]
49:[0]
50:[0 1]
51:[0]
52:[0]
53:[0]
54:[0]
55:[0 1]
56:[0]
57:[0]
58:[0]
59:[0]
60:[0 1]
61:[0]
62:[0]
63:[0]
64:[0]
65:[0]
66:[0]
67:[0]
68:[0]
69:[0]
70:[0]
71:[0]
72:[0]
73:[0]
74:[0]
75:[0 1]
76:[0 1]
77:[0]
78:[0 1]
79:[0]
80:[0]
81:[0]
82:[0 1]
83:[0]
84:[0]
85:[0]
86:[0]
87:[0]
88:[0]
89:[0]
90:[0]
91:[0 1]
92:[0 1]
93:[0]
94:[0]
95:[0 1]
96:[0 1]
97:[0 1]
98:[0 1]
99:[0]
100:[0]
101:[0]
102:[0]
103:[0 1]
104:[0]


In [30]:
train[(train.building_id == 8) & (train.meter == 0) & (train.meter_reading != 0)].head(10)

,building_id,meter,timestamp,meter_reading,exception
6509944,8,0,2016-05-01 19:00:00,749.000,0
7576731,8,0,2016-05-20 18:00:00,270.294,0
7579078,8,0,2016-05-20 19:00:00,348.788,0
7581424,8,0,2016-05-20 20:00:00,367.217,0
7583771,8,0,2016-05-20 21:00:00,366.193,0
7586117,8,0,2016-05-20 22:00:00,342.645,0
7588462,8,0,2016-05-20 23:00:00,304.763,0
7590806,8,0,2016-05-21 00:00:00,308.517,0
7593153,8,0,2016-05-21 01:00:00,323.875,0
7595499,8,0,2016-05-21 02:00:00,309.882,0


In [31]:
# 2016-05-20 18:00:00 为恢复供电时间， 53有点特别，也许全年无消耗

In [32]:
number = 0
for i in range(105):
    number += train[(train.building_id == i) & (train.timestamp < '2016-05-20 18:00:00')].shape[0]
    print(number)

3378
6756
10134
13512
16890
20268
23646
28977
32355
37686
41064
44442
47820
53151
58482
63813
67191
70569
73947
77325
80703
84081
87459
90837
94215
97593
100971
104349
108618
110628
115959
121289
124667
128045
131423
134801
138179
141557
144935
148313
151649
155027
158405
163736
167114
170491
173869
177247
179213
182591
187922
191300
194678
196156
199534
204865
208243
211621
214999
218377
223708
227086
230464
233842
237220
240598
243976
247354
250732
254110
257488
260866
264244
267622
271000
276331
281662
285040
290371
293749
297127
300505
305836
309214
312592
315970
319348
322726
326104
329482
332860
338191
343522
346900
350278
355609
360940
366220
369961
373339
376717
380095
383473
388735
392113


In [33]:
number = 0
for i in range(105):
    number += test[(test.building_id == i) & ((test.timestamp < '2017-05-20 18:00:00') | ((test.timestamp > '2017-12-31 23:00:00') & (test.timestamp < '2018-05-20 18:00:00')))].shape[0]
print(number)

865332
